In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import datacompy
import os, sys
import numpy as np
import openpyxl

# narzedzia
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    cross_val_predict,
    learning_curve,
    RepeatedStratifiedKFold,
    GridSearchCV,
)
from collections import Counter
from sklearn.metrics import classification_report, confusion_matrix, silhouette_score, accuracy_score
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from scipy import stats
from joblib import dump, load
from openpyxl import load_workbook
import xlsxwriter

# modele
from xgboost import XGBClassifier, XGBRFClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# methods
from imblearn.under_sampling import ClusterCentroids, NearMiss
from scipy.optimize import differential_evolution
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.neighbors import LocalOutlierFactor
from sklearn.inspection import permutation_importance

from imblearn.over_sampling import SMOTE, BorderlineSMOTE
from ctgan import CTGAN

Please note that you are missing the optional dependency: fugue. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: snowflake. If you need to use this functionality it must be installed.
Please note that you are missing the optional dependency: spark. If you need to use this functionality it must be installed.
Python 3.12 and above currently is not supported by Spark and Ray. Please note that some functionality will not work and currently is not supported.


### Read data

In [2]:
mix_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_HDBSCAN_cos.csv")
mix_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_HDBSCAN_NN.csv")
mix_KM_centroids = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_KM_centroids.csv")
mix_KM_cos_mal = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_KM_cos_mal.csv")
mix_KM_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_KM_cos.csv")
mix_KM_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_KM_nn.csv")
mix_KM_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_kmeans&hdbscan_cos.csv")
mix_KM_HDBSCAN_cos_mal = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_kmeans&hdbscan_cos&mal.csv")
mix_KM_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\mix_kmeans&hdbscan_nn.csv")

borderline_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_HDBSCAN_cos.csv")
borderline_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_HDBSCAN_NN.csv")
borderline_KM_centroids = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_KM_centroids.csv")
borderline_KM_cos_mal = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_KM_cos_mal.csv")
borderline_KM_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_KM_cos.csv")
borderline_KM_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_KM_nn.csv")
borderline_KM_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_kmeans&hdbscan_cos.csv")
borderline_KM_HDBSCAN_cos_mal = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_kmeans&hdbscan_cos&mal.csv")
borderline_KM_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\borderline_kmeans&hdbscan_nn.csv")    

smote_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_HDBSCAN_cos.csv")
smote_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_HDBSCAN_NN.csv")
smote_KM_centroids = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_KM_centroids.csv")
smote_KM_cos_mal = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_KM_cos_mal.csv")
smote_KM_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_KM_cos.csv")
smote_KM_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_KM_nn.csv")
smote_KM_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_kmeans&hdbscan_cos.csv")
smote_KM_HDBSCAN_cos_mal = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_kmeans&hdbscan_cos&mal.csv")
smote_KM_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\smote_kmeans&hdbscan_nn.csv")

GAN_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_HDBSCAN_cos.csv")
GAN_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_HDBSCAN_NN.csv")
GAN_KM_centroids = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_KM_centroids.csv")
GAN_KM_cos_mal = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_KM_cos_mal.csv")  
GAN_KM_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_KM_cos.csv")
GAN_KM_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_KM_nn.csv")
GAN_KM_HDBSCAN_cos = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_kmeans&hdbscan_cos.csv")
GAN_KM_HDBSCAN_cos_mal = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_kmeans&hdbscan_cos&mal.csv")
GAN_KM_HDBSCAN_nn = pd.read_csv("D:\\ml\\undersampling_data\\data\\ssh\\reduced\\GAN_kmeans&hdbscan_nn.csv")    


### Dictionary


In [3]:
data = {}

data["mix_HDBSCAN_cos"] = mix_HDBSCAN_cos   
data["mix_HDBSCAN_nn"] = mix_HDBSCAN_nn
data["mix_KM_centroids"] = mix_KM_centroids
data["mix_KM_cos_mal"] = mix_KM_cos_mal
data["mix_KM_cos"] = mix_KM_cos
data["mix_KM_nn"] = mix_KM_nn
data["mix_KM_HDBSCAN_cos"] = mix_KM_HDBSCAN_cos
data["mix_KM_HDBSCAN_cos_mal"] = mix_KM_HDBSCAN_cos_mal
data["mix_KM_HDBSCAN_nn"] = mix_KM_HDBSCAN_nn

data["borderline_HDBSCAN_cos"] = borderline_HDBSCAN_cos
data["borderline_HDBSCAN_nn"] = borderline_HDBSCAN_nn
data["borderline_KM_centroids"] = borderline_KM_centroids
data["borderline_KM_cos_mal"] = borderline_KM_cos_mal
data["borderline_KM_cos"] = borderline_KM_cos
data["borderline_KM_nn"] = borderline_KM_nn
data["borderline_KM_HDBSCAN_cos"] = borderline_KM_HDBSCAN_cos
data["borderline_KM_HDBSCAN_cos_mal"] = borderline_KM_HDBSCAN_cos_mal
data["borderline_KM_HDBSCAN_nn"] = borderline_KM_HDBSCAN_nn

data["smote_HDBSCAN_cos"] = smote_HDBSCAN_cos
data["smote_HDBSCAN_nn"] = smote_HDBSCAN_nn
data["smote_KM_centroids"] = smote_KM_centroids
data["smote_KM_cos_mal"] = smote_KM_cos_mal
data["smote_KM_cos"] = smote_KM_cos
data["smote_KM_nn"] = smote_KM_nn
data["smote_KM_HDBSCAN_cos"] = smote_KM_HDBSCAN_cos
data["smote_KM_HDBSCAN_cos_mal"] = smote_KM_HDBSCAN_cos_mal
data["smote_KM_HDBSCAN_nn"] = smote_KM_HDBSCAN_nn

data["GAN_HDBSCAN_cos"] = GAN_HDBSCAN_cos
data["GAN_HDBSCAN_nn"] = GAN_HDBSCAN_nn
data["GAN_KM_centroids"] = GAN_KM_centroids
data["GAN_KM_cos_mal"] = GAN_KM_cos_mal
data["GAN_KM_cos"] = GAN_KM_cos
data["GAN_KM_nn"] = GAN_KM_nn
data["GAN_KM_HDBSCAN_cos"] = GAN_KM_HDBSCAN_cos
data["GAN_KM_HDBSCAN_cos_mal"] = GAN_KM_HDBSCAN_cos_mal
data["GAN_KM_HDBSCAN_nn"] = GAN_KM_HDBSCAN_nn

### Distribiution

In [4]:
output_xlsx = "D:\\ml\\undersampling_data\\reports\\ssh\\distribiution\\distribiution.xlsx"
output_txt = "D:\\ml\\undersampling_data\\reports\\ssh\\distribiution\\distribiution.txt"


all_results = pd.DataFrame()

for name, data in data.items():
    
    data = pd.DataFrame(data)
    results = data["source"].value_counts(normalize=True) * 100
    results.name = name
    print(f"Data: {name}")
    print(f"Distribiution:\n{results}")
    
    with open(output_txt, "a") as f:
        f.write(f"Data: {name}\n")
        f.write(f"Distribiution:\n{results}\n\n")
    
    # Append results to the combined DataFrame
    all_results = pd.concat([all_results, results], axis=1)
    
with pd.ExcelWriter(output_xlsx, engine='openpyxl', mode='a', if_sheet_exists='replace') as writer:
    all_results.to_excel(writer, sheet_name='Distribution', index=True)


    
#compare collect and sum_data_NM and copy column source to collect, if rows are the same
""" collect = collect.merge(sum_data_NM, on=collect.columns.tolist(), how='left', indicator=True)
collect = collect.drop(columns=["_merge"])
collect = collect.drop_duplicates()
print(collect)
print(collect["source"].value_counts()) """

Data: mix_HDBSCAN_cos
Distribiution:
source
original            65.231788
smote               14.900662
borderline smote    13.907285
gan                  5.960265
Name: mix_HDBSCAN_cos, dtype: float64
Data: mix_HDBSCAN_nn
Distribiution:
source
original            65.231788
borderline smote    14.569536
smote               14.238411
gan                  5.960265
Name: mix_HDBSCAN_nn, dtype: float64
Data: mix_KM_centroids
Distribiution:
source
original    65.231788
centroid    34.768212
Name: mix_KM_centroids, dtype: float64
Data: mix_KM_cos_mal
Distribiution:
source
original            65.231788
gan                 17.880795
borderline smote     8.940397
smote                7.947020
Name: mix_KM_cos_mal, dtype: float64
Data: mix_KM_cos
Distribiution:
source
original            65.231788
gan                 18.543046
borderline smote     9.271523
smote                6.953642
Name: mix_KM_cos, dtype: float64
Data: mix_KM_nn
Distribiution:
source
original            65.231788
gan       

' collect = collect.merge(sum_data_NM, on=collect.columns.tolist(), how=\'left\', indicator=True)\ncollect = collect.drop(columns=["_merge"])\ncollect = collect.drop_duplicates()\nprint(collect)\nprint(collect["source"].value_counts()) '